In [ ]:
!pip install -q langchain-qdrant
!pip install -q qdrant-client
!pip install -q langchain
!pip install -q langchain-community
!pip install -q unstructured
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q langchain_huggingface

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from google.colab import userdata
import os
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain.docstore.document import Document


In [ ]:
url = "https://1390ac05-4421-4fc6-8798-05032e7abad2.us-east4-0.gcp.cloud.qdrant.io"
api = "02n1zyvj8zLx0xaAPk-_pbVSVIq4sFXVUbfMWzl3mCHJdkpTHMn3Cw"

In [ ]:
embedding_model = HuggingFaceEmbeddings(
     model_name="dangvantuan/vietnamese-document-embedding",
     model_kwargs={"trust_remote_code": True}  # Add this parameter
)

In [ ]:
from qdrant_client import QdrantClient
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(Path("./.env"))

embedding_model = HuggingFaceEmbeddings(
    model_name="dangvantuan/vietnamese-document-embedding",
    model_kwargs={"trust_remote_code": True}
    )

client = QdrantClient(
     url=url,
     api_key=api,
     prefer_grpc=False
)

vector_store = QdrantVectorStore(
               client=client,
               collection_name="TDTU",
               embedding=embedding_model
          )
query = "Chỉ tiêu tuyển sinh TDTU 2022"
docs = vector_store.similarity_search(query, k= 3)
print(docs)



[Document(metadata={'source': 'Admission\\TDTU\\2022_tuyển sinh.md', '_id': '89cacdb3-ee4b-41ee-9105-7f6923d29fec', '_collection_name': 'TDTU'}, page_content='PHƯƠNG ÁN TUYỂN SINH ĐẠI HỌC NĂM 2022\n\nTrường Đại học Tôn Đức Thắng (TDTU) dự kiến tuyển sinh khoảng 6.500 chỉ tiêu trình độ đại học năm 2022; bao gồm 40 ngành Chương trình tiêu chuẩn, 17 ngành Chương trình chất lượng cao, 12 ngành Chương trình đại học bằng tiếng Anh, 07 ngành chương trình học tại phân hiệu Khánh Hòa và 11 ngành Chương trình liên kết đào tạo quốc tế. Nhà trường dự kiến triển khai tuyển sinh đại học năm 2022 theo 05 phương thức:\n\nPhương thức 1: Xét tuyển theo kết quả quá trình học tập THPT\n\nPhương thức 2. Xét tuyển theo kết quả thi tốt nghiệp THPT năm 2022\n\nPhương thức 3. Ưu tiên xét tuyển theo quy định của TDTU\n\nPhương thức 4: Xét tuyển thẳng, ưu tiên xét tuyển theo Quy chế tuyển sinh của Bộ GD&ĐT\n\nPhương thức 5: Xét tuyển theo kết quả bài thi đánh giá năng lực của Đại học Quốc gia Thành phố Hồ Chí Mi

In [ ]:
import google.generativeai as genai

class BaseRetrievalStrategy:
     def __init__(self, llm = None):
          self.embeddings = embedding_model
          self.llm = llm or genai.GenerativeModel(model_name= "gemini-1.5-flash-8b")
          self.classifier = QueryRouter()

     def retrieve(self, query, k=3):
          return self.db.similarity_search(query, k=k)

class UniversityRetrievalStrategy(BaseRetrievalStrategy):

     def retrieve(self, query: str, k = 3)-> list:
          university = self.classifier.UniversityRouting(query).university_code
          print(f'Truy vấn thuộc về trường: {university}')

          client = QdrantClient(
               url=os.getenv('qdrant_url2'),
               api_key=os.getenv('qdrant_api2'),
               prefer_grpc=False
          )

          vector_store = QdrantVectorStore(
               client=client,
               collection_name=str(university),
               embedding=embedding_model
          )
          return vector_store.similarity_search(query, k)
if __name__ == "__main__":

     retriever = UniversityRetrievalStrategy()
     query = "Chỉ tiêu tuyển sinh TDTU 2022"
     docs = retriever.retrieve(query, k= 3)
     print(docs)

## **Load from Qdrant**

In [ ]:
from qdrant_client import QdrantClient

client = QdrantClient(
    url=url,
    api_key=api,
)

In [ ]:
base_folder = "/content/drive/MyDrive/Data/Test/Admission"


for folder_name in os.listdir(base_folder):
    documents = []
    scroll_result = client.scroll(
        collection_name=folder_name,
        limit= 500
    )

    for point in scroll_result[0]:

        doc_entry = {
            'id': point.id,
            **point.payload  # Unpack all payload fields
        }
        documents.append(doc_entry)

    df = pd.DataFrame(documents)
    output_file = "/content/drive/MyDrive/Data/SemanticChunks/" + folder_name + ".xlsx"
    df.to_excel(output_file, index=False)
    print(f"Exported {len(documents)} documents to {output_file}")



Exported 74 documents to /content/drive/MyDrive/Data/SemanticChunks/HCMUE.xlsx
Exported 59 documents to /content/drive/MyDrive/Data/SemanticChunks/HCMUTE.xlsx
Exported 77 documents to /content/drive/MyDrive/Data/SemanticChunks/NTTU.xlsx
Exported 16 documents to /content/drive/MyDrive/Data/SemanticChunks/PNTU.xlsx
Exported 43 documents to /content/drive/MyDrive/Data/SemanticChunks/BKU.xlsx
Exported 24 documents to /content/drive/MyDrive/Data/SemanticChunks/FTU2.xlsx
Exported 78 documents to /content/drive/MyDrive/Data/SemanticChunks/TDTU.xlsx
Exported 65 documents to /content/drive/MyDrive/Data/SemanticChunks/OU.xlsx
Exported 23 documents to /content/drive/MyDrive/Data/SemanticChunks/FPT.xlsx
Exported 27 documents to /content/drive/MyDrive/Data/SemanticChunks/HCMUS.xlsx
Exported 47 documents to /content/drive/MyDrive/Data/SemanticChunks/UEH.xlsx
Exported 50 documents to /content/drive/MyDrive/Data/SemanticChunks/UMP.xlsx
Exported 28 documents to /content/drive/MyDrive/Data/SemanticChunk

# **Chunking and Store with each university is a collection**

In [ ]:
base_folder = "/content/drive/MyDrive/Data/Test/Admission"


for folder_name in os.listdir(base_folder):
    all_documents = []
    folder_path = os.path.join(base_folder, folder_name)
    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder_name}")
        filename = folder_name + ".xlsx"
        file_path = os.path.join(folder_path, filename)
        print(file_path)
        df = pd.read_excel(file_path)

        for _, row in df.iterrows():
            doc = Document(
                page_content=row['text'],
                metadata={'source': row['file name']}
            )
            all_documents.append(doc)
        vector_store = Qdrant.from_documents(
          documents=all_documents,
          embedding=embedding_model,
          url="https://1390ac05-4421-4fc6-8798-05032e7abad2.us-east4-0.gcp.cloud.qdrant.io:6333",
          api_key="02n1zyvj8zLx0xaAPk-_pbVSVIq4sFXVUbfMWzl3mCHJdkpTHMn3Cw",
          prefer_grpc=False,
          collection_name=folder_name,
        )
    print(f"Uploaded documents to Qdrant collection: {folder_name}")



Processing folder: VLU
/content/drive/MyDrive/Data/Test/Admission/VLU/VLU.xlsx
Uploaded documents to Qdrant collection: VLU


In [ ]:
client = QdrantClient(
              url="https://1390ac05-4421-4fc6-8798-05032e7abad2.us-east4-0.gcp.cloud.qdrant.io:6333",
              api_key="02n1zyvj8zLx0xaAPk-_pbVSVIq4sFXVUbfMWzl3mCHJdkpTHMn3Cw",
               prefer_grpc=True
          )

vector_store = QdrantVectorStore(
               client=client,
               collection_name="VLU",
               embedding=embedding_model
          )

In [ ]:
query = "Mã ngành Điều dưỡng "
print(vector_store.similarity_search(query, 3))

[Document(metadata={'source': 'Giới thiệu chung DH Văn Lang.txt', '_id': '0adb0fbc-b0aa-4557-9348-05ed15eb2325', '_collection_name': 'VLU'}, page_content='20. Điều dưỡng\nĐiều dưỡng có vai trò quan trọng trong hệ thống y tế, thực hiện chăm sóc trực tiếp cho người bệnh tại bệnh viện và cộng đồng, đồng thời phối hợp với các nghề khác để cung cấp dịch vụ chăm sóc sức khỏe toàn diện.\na. Mã ngành:\n7720301\nb. Văn bằng:\nCử nhân Điều dưỡng\nc. Tổ hợp môn:\n•\tB00: Toán – Hóa – Sinh\n•\tC08: Văn – Hóa – Sinh\n•\tD07: Toán – Hóa – Anh\n•\tD08: Toán – Sinh – Anh\nd. Thời gian đào tạo:\n3,5 - 4 năm\n\n21. Đông Phương học (Hàn Quốc học, Nhật Bản học, Trung Quốc học)\nNgành Đông Phương học cung cấp kiến thức chuyên sâu và toàn diện về các quốc gia khu vực Đông Á như Hàn Quốc, Trung Quốc và Nhật Bản, giúp sinh viên có khả năng làm việc trong các tổ chức quốc tế, doanh nghiệp, và nhiều lĩnh vực khác như ngoại giao, du lịch, giáo dục, truyền thông.\na. Mã ngành:\n7310608\nb. Văn bằng:\nCử nhân Đông